<a href="https://colab.research.google.com/github/Rishabhjaiswal7/Tiny_Coders/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [1]:
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr
import os

# Create output folder
os.makedirs("outputs", exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("🚀 Running on:", device)

# Load SD v1.5
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

pipe.enable_attention_slicing()

try:
    pipe.enable_xformers_memory_efficient_attention()
    print("✅ xFormers Enabled")
except:
    print("ℹ xFormers not available")

from torch.cuda.amp import autocast   # <-- SAFE + NO WARNINGS

def generate_image(prompt: str):
    if not prompt.strip():
        return None, "⚠ Please enter a prompt."

    if device == "cuda":
        with autocast():
            image = pipe(prompt, guidance_scale=7.0).images[0]
    else:
        image = pipe(prompt, guidance_scale=7.0).images[0]

    img_path = "outputs/generated.png"
    image.save(img_path)

    return image, f"Image generated for: {prompt}"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Simple AI Image Generator (Stable Diffusion v1.5)")
    prompt = gr.Textbox(label="Enter your prompt here:")
    generate_btn = gr.Button("Generate Image")

    output_image = gr.Image(label="Generated Image")
    output_text = gr.Textbox(label="Status Message")

    generate_btn.click(generate_image, inputs=prompt, outputs=[output_image, output_text])

if __name__ == "__main__":
    demo.launch()



🚀 Running on: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


ℹ xFormers not available
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ea3930290f12db779b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
def generate_image(prompt: str):
    if not prompt.strip():
        return None, "⚠ Please enter a prompt."

    start_time = time.time()

    # GPU autocast
    if device == "cuda":
        with autocast():
            image = pipe(prompt, guidance_scale=7.0).images[0]
    else:
        image = pipe(prompt, guidance_scale=7.0).images[0]

    end_time = time.time()
    infer_time = round(end_time - start_time, 2)
    total_time = round(end_time - start_time + model_load_time, 2)

    # Save output
    img_path = "outputs/generated.png"
    image.save(img_path)

    # --- Quality Metrics in Percent ---
    width, height = image.size
    resolution = f"{width} x {height}"
    base_pixels = 512 * 512
    quality_percent = min(100, (width * height) / base_pixels * 100)
    quality_percent = round(quality_percent, 2)

    # --- GPU / RAM usage ---
    if device == "cuda":
        vram_used = torch.cuda.memory_allocated() / (1024 ** 2)
        vram_text = f"{vram_used:.2f} MB"
    else:
        vram_text = "N/A (CPU)"

    ram_used = psutil.Process().memory_info().rss / (1024 ** 2)
    ram_text = f"{ram_used:.2f} MB"

    status = f"""
📊 **Performance Metrics**
----------------------------------
🖥 **Device:** {device.upper()}
⏳ **Model Load Time:** {model_load_time}s
⚡ **Generation Time:** {infer_time}s
⏱ **Total Execution Time:** {total_time}s

💾 **VRAM Used:** {vram_text}
🧠 **RAM Used:** {ram_text}

🖼 **Resolution:** {resolution}
🌟 **Quality Score:** {quality_percent}%

📝 **Prompt:** {prompt}
"""

    return image, status


## **Introduction to Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab where you will get a quick introduction to the Python programming language and the environment used for the course's exercises: Colab.

Colab is a Python development environment that runs in the browser using Google Cloud.

For example, to print "Hello World", just hover the mouse over [ ] and press the play button to the upper left. Or press shift-enter to execute.

In [3]:
print("Hello World")

Hello World


## Functions, Conditionals, and Iteration
Let's create a Python function, and call it from a loop.

In [4]:
def HelloWorldXY(x, y):
  if (x < 10):
    print("Hello World, x was < 10")
  elif (x < 20):
    print("Hello World, x was >= 10 but < 20")
  else:
    print("Hello World, x was >= 20")
  return x + y

for i in range(8, 25, 5):  # i=8, 13, 18, 23 (start, stop, step)
  print("--- Now running with i: {}".format(i))
  r = HelloWorldXY(i,i)
  print("Result from HelloWorld: {}".format(r))

--- Now running with i: 8
Hello World, x was < 10
Result from HelloWorld: 16
--- Now running with i: 13
Hello World, x was >= 10 but < 20
Result from HelloWorld: 26
--- Now running with i: 18
Hello World, x was >= 10 but < 20
Result from HelloWorld: 36
--- Now running with i: 23
Hello World, x was >= 20
Result from HelloWorld: 46


In [5]:
print(HelloWorldXY(1,2))

Hello World, x was < 10
3


Easy, right?

If you want a loop starting at 0 to 2 (exclusive) you could do any of the following

In [6]:
print("Iterate over the items. `range(2)` is like a list [0,1].")
for i in range(2):
  print(i)

print("Iterate over an actual list.")
for i in [0,1]:
  print(i)

print("While works")
i = 0
while i < 2:
  print(i)
  i += 1

Iterate over the items. `range(2)` is like a list [0,1].
0
1
Iterate over an actual list.
0
1
While works
0
1


In [7]:
print("Python supports standard key words like continue and break")
while True:
  print("Entered while")
  break

Python supports standard key words like continue and break
Entered while


## Numpy and lists
Python has lists built into the language.
However, we will use a library called numpy for this.
Numpy gives you lots of support functions that are useful when doing Machine Learning.

Here, you will also see an import statement. This statement makes the entire numpy package available and we can access those symbols using the abbreviated 'np' syntax.

In [8]:
import numpy as np  # Make numpy available using np.

# Create a numpy array, and append an element
a = np.array(["Hello", "World"])
a = np.append(a, "!")
print("Current array: {}".format(a))
print("Printing each element")
for i in a:
  print(i)

print("\nPrinting each element and their index")
for i,e in enumerate(a):
  print("Index: {}, was: {}".format(i, e))

Current array: ['Hello' 'World' '!']
Printing each element
Hello
World
!

Printing each element and their index
Index: 0, was: Hello
Index: 1, was: World
Index: 2, was: !


In [9]:
print("\nShowing some basic math on arrays")
b = np.array([0,1,4,3,2])
print("Max: {}".format(np.max(b)))
print("Average: {}".format(np.average(b)))
print("Max index: {}".format(np.argmax(b)))


Showing some basic math on arrays
Max: 4
Average: 2.0
Max index: 2


In [10]:
print("\nYou can print the type of anything")
print("Type of b: {}, type of b[0]: {}".format(type(b), type(b[0])))


You can print the type of anything
Type of b: <class 'numpy.ndarray'>, type of b[0]: <class 'numpy.int64'>


In [11]:
print("\nUse numpy to create a [3,3] dimension array with random number")
c = np.random.rand(3, 3)
print(c)


Use numpy to create a [3,3] dimension array with random number
[[0.95325556 0.43429207 0.94959842]
 [0.89537191 0.49798026 0.41441635]
 [0.21130588 0.84724812 0.03860306]]


In [12]:
print("\nYou can print the dimensions of arrays")
print("Shape of a: {}".format(a.shape))
print("Shape of b: {}".format(b.shape))
print("Shape of c: {}".format(c.shape))
print("...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists")


You can print the dimensions of arrays
Shape of a: (3,)
Shape of b: (5,)
Shape of c: (3, 3)
...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists


## Colab Specifics

Colab is a virtual machine you can access directly. To run commands at the VM's terminal, prefix the line with an exclamation point (!).


In [13]:
print("\nDoing $ls on filesystem")
!ls -l
!pwd


Doing $ls on filesystem
total 8
drwxr-xr-x 2 root root 4096 Nov 22 12:16 outputs
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data
/content


In [14]:
print("Install numpy")  # Just for test, numpy is actually preinstalled in all Colab instances
!pip install numpy

Install numpy


**Exercise**

Create a code cell underneath this text cell and add code to:


*   List the path of the current directory (pwd)
* Go to / (cd) and list the content (ls -l)

In [15]:
!pwd
!cd /
!ls -l
print("Hello")

/content
total 8
drwxr-xr-x 2 root root 4096 Nov 22 12:16 outputs
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data
Hello


All usage of Colab in this course is completely free or charge. Even GPU usage is provided free of charge for some hours of usage every day.

**Using GPUs**
* Many of the exercises in the course executes more quickly by using GPU runtime: Runtime | Change runtime type | Hardware accelerator | GPU

**Some final words on Colab**
*   You execute each cell in order, you can edit & re-execute cells if you want
*   Sometimes, this could have unintended consequences. For example, if you add a dimension to an array and execute the cell multiple times, then the cells after may not work. If you encounter problem reset your environment:
  *   Runtime -> Restart runtime... Resets your Python shell
  *   Runtime -> Restart all runtimes... Will reset the Colab image, and get you back to a 100% clean environment
* You can also clear the output in the Colab by doing: Edit -> Clear all outputs
* Colabs in this course are loaded from GitHub. Save to your Google Drive if you want a copy with your code/output: File -> Save a copy in Drive...

**Learn More**
*   Check out [this](https://www.youtube.com/watch?v=inN8seMm7UI&list=PLQY2H8rRoyvwLbzbnKJ59NkZvQAW9wLbx&index=3) episode of #CodingTensorFlow, and don't forget to subscribe to the YouTube channel ;)
